In [3]:
import requests
import string

from utils.get_data import get_user, get_project, get_all_projects, get_all_users
from utils.metric_utils import intersection_metric, iou_metric
from utils.similarity_utils import cosine_similarity
from utils.text_utils import embedd_representation, get_user_roles_desc, get_user_text_desc, get_user_skill_desc, get_project_text_desc
from tqdm import tqdm
import pickle
import pandas as pd
import numpy as np
from scipy.spatial import distance
from typing import Dict, Any, List

In [4]:
test_parquet = pd.read_parquet('/Users/andrey.krotkikh/Documents/embedds.parquet')

In [108]:
get_user(1)

{'_id': '1',
 'firstName': 'Иван',
 'secondName': 'Иванов',
 'lastName': 'Иванович',
 'birthDate': '26.10.1972',
 'country': 'Россия',
 'city': 'Москва',
 'citizenship': 'Российское',
 'gender': 'Мужской',
 'contacts': {'phone': '+95159195192',
  'email': 'ivanov@yandex.ru',
  'vk': '@ivanov',
  'telegram': '@ivanov',
  'skype': 'ivanov'},
 'education': {'startDate': '10.10.1990',
  'startEnd': '10.10.1996',
  'speciality': 'Компьютерные системы',
  'universityName': 'МГУ'},
 'jobInformation': {'employment': 'На себя',
  'jobExperience': '20',
  'achievements': ['Победа в хакатоне 2021',
   'Победа в хакатоне 2006',
   'Работал в СБЕР']},
 'skillInformation': {'categories': ['Капитан', 'Teamlead', 'Backend'],
  'skills': ['Excel', 'C#', '.net', 'Презентация', 'C++']},
 'intellectualProperty': {'isIntellectualPropertyAuthor': True,
  'numberOfDocument': '51295195129512'},
 'companyInformation': {'isCompanyOwner': True,
  'companyINN': '52151295912959'},
 'hackathon': {'countHackathons':

In [9]:
def search_users_for_user(id: int, input_df: pd.DataFrame) -> Dict[int, Dict[str, float]]:
    interests = get_user(id)['interestedTags']
    desc = get_user(id)['aboutDescription']

    all_users = get_all_users()

    result_dict = {}

    for user in all_users:
        if int(user['_id']) != id:
            interests_metric = intersection_metric(interests, user['interestedTags'])
            semantic_metric = cosine_similarity(embedd_representation(desc, input_df), embedd_representation(user['aboutDescription'], input_df))
            result_dict[user['_id']] = {"Interests": interests_metric, "Semantic": semantic_metric}

    return result_dict

In [10]:
search_users_for_user(1, test_parquet)

{'2': {'Interests': 0, 'Semantic': 0.7239373922348022},
 '3': {'Interests': 0, 'Semantic': 0.7435581088066101},
 '4': {'Interests': 0, 'Semantic': 0.8428709506988525},
 '5': {'Interests': 1, 'Semantic': 0.7765620553897354},
 '6': {'Interests': 0, 'Semantic': 0.8440775997855627},
 '7': {'Interests': 1, 'Semantic': 0.9152721743055132},
 '8': {'Interests': 1, 'Semantic': 0.9152721743055132},
 '9': {'Interests': 1, 'Semantic': 0.9120366041283666},
 '10': {'Interests': 1, 'Semantic': 0.7998495697975159}}

In [11]:
get_user(1)['aboutDescription']

'Я уже опытный разработчик, хочу участвовать в разработке социальной сети. Есть несколько крутых идей, с помощью которых мы изменим мир!'

In [13]:
get_user(8)['aboutDescription']

'Студент московского вуза, работаю разработчиком java back. В среднесрочных целях - переезд в другую страну с получением там работы, в связи с этим много работаю и учусь Помимо тех знаний интересуюсь различными направлениями от английского и психологии до нейробиологии и нутрициологии, в основном всем тем, что помогает эффективнее учится и лучше себя чувствовать. Год коммерческого опыта в разработке на java. Работаю в альфа-банке, занимаюсь разработкой бэкенда для мобильного приложения На начальном уровне знаю c++, python, машинное обучение. В моих планах развиваться как it специалист, а это происходит быстрее при обмене опытом, если вам интересно, то можем попробовать вместе начать какой нибудь проект учебный/коммерческий или просто иметь контакты друг друга и обращаться за помощью'